# Download Yellow 2025-10 data on local

In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-14 16:37:32--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.97, 52.85.39.65, 52.85.39.153, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: 'yellow_tripdata_2024-10.parquet'

     0K .......... .......... .......... .......... ..........  0%  788K 80s
    50K .......... .......... .......... .......... ..........  0%  730K 83s
   100K .......... .......... .......... .......... ..........  0% 1.80M 66s
   150K .......... .......... .......... .......... ..........  0% 1.84M 58s
   200K .......... .......... .......... .......... ..........  0% 1.80M 53s
   250K .......... .......... .......... .......... ..........  0% 2.64M 48s
   300K .......... .......... .......... .......... ..........  0% 1.

In [4]:
!ls -ah

.
..
.ipynb_checkpoints
class_examples.ipynb
homework.ipynb
homework.md
taxi_zone_lookup.csv
yellow_tripdata_2024-10.parquet
zones


In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

# Question 2: Yellow October 2024
Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [7]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [8]:
df = df.repartition(4)

In [9]:
df.write.parquet('yellow_tripdata_repartition')

In [11]:
!ls -lh

total 62M
-rw-r--r-- 1 swang 197121 7.6K Mar 14 16:37 class_examples.ipynb
-rw-r--r-- 1 swang 197121 112K Mar 14 16:44 homework.ipynb
-rw-r--r-- 1 swang 197121 2.0K Mar 14 16:33 homework.md
-rw-r--r-- 1 swang 197121  13K Mar 14 15:04 taxi_zone_lookup.csv
-rw-r--r-- 1 swang 197121  62M Dec 18 13:21 yellow_tripdata_2024-10.parquet
drwxr-xr-x 1 swang 197121    0 Mar 14 16:45 yellow_tripdata_repartition
drwxr-xr-x 1 swang 197121    0 Mar 14 15:39 zones


In [14]:
!du -sh yellow_tripdata_repartition

91M	yellow_tripdata_repartition


In [15]:
!ls -lh yellow_tripdata_repartition

total 90M
-rw-r--r-- 1 swang 197121   0 Mar 14 16:45 _SUCCESS
-rw-r--r-- 1 swang 197121 23M Mar 14 16:45 part-00000-8f3eaaef-6843-4c0c-ada9-69dd0883d996-c000.snappy.parquet
-rw-r--r-- 1 swang 197121 23M Mar 14 16:45 part-00001-8f3eaaef-6843-4c0c-ada9-69dd0883d996-c000.snappy.parquet
-rw-r--r-- 1 swang 197121 23M Mar 14 16:45 part-00002-8f3eaaef-6843-4c0c-ada9-69dd0883d996-c000.snappy.parquet
-rw-r--r-- 1 swang 197121 23M Mar 14 16:45 part-00003-8f3eaaef-6843-4c0c-ada9-69dd0883d996-c000.snappy.parquet


# Question 3: Count records

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [5]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [6]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_timestamp, lit

df.withColumn('pickup_date', F.to_date(col('tpep_pickup_datetime'))) \
    .withColumn('dropoff_date', F.to_date(col('tpep_dropoff_datetime'))) \
    .filter(col('pickup_date') == lit("2024-10-15").cast("date")) \
    .filter(col('dropoff_date') == lit("2024-10-15").cast("date")).count()

127993

# Question 4: Longest trip
What is the length of the longest trip in the dataset in hours?

In [7]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [17]:
from pyspark.sql import types
from pyspark.sql.functions import unix_timestamp, round, max

df.withColumn(
    "duration_hours",
    round((unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600, 2)) \
    .select(max('duration_hours')).show()

+-------------------+
|max(duration_hours)|
+-------------------+
|             162.62|
+-------------------+



# Question 5: User Interface
Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- [x] 4040
- 8080

# Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?


In [20]:
df_zone = spark.read \
    .option('header', 'true') \
    .csv('taxi_zone_lookup.csv')

df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [24]:
from pyspark.sql.functions import count, desc

df_join = df.join(df_zone, df.PULocationID == df_zone.LocationID)

df_join.groupBy('Zone').agg(count("*").alias("count")).orderBy("count").show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|       Rikers Island|    2|
|       Arden Heights|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Charleston/Totten...|    4|
|   Rossville/Woodrow|    4|
|       West Brighton|    4|
|Eltingville/Annad...|    4|
|       Port Richmond|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|Heartland Village...|    7|
|     Mariners Harbor|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|       Broad Channel|   10|
|New Dorp/Midland ...|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

